In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/carlo/workspace/networkqit/')
import numpy as np
import networkqit as nq
import networkx as nx
from numpy.random import random as rand
from networkqit import graph_laplacian as GL
from scipy.linalg import eigvalsh
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

Astar = np.loadtxt('/home/carlo/workspace/networkqit/notebooks/ring_3_4_5_6_7_8.adj',delimiter=',')
G = nx.from_numpy_array(Astar)
k = Astar.sum(axis=0).astype(int)
Lstar = nq.graph_laplacian(Astar)
pstar = nx.density(G)
N = len(G.nodes())
M=nq.UBCM(N=N)

optimizer = nq.MLEOptimizer(A=Astar,x0=np.random.random(N))
sol = optimizer.runfsolve(model='UBCM')

In [ ]:
beta_range = np.logspace(2,-4,500)

In [ ]:
# initialize from the solution
#optimizer = nq.ExpectedModelOptimizer(A=Astar,beta_range=beta_range,x0=np.random.random(N))
#optimizer.setup(expected_adj_fun=M,expected_lapl_grad_fun=M.expected_laplacian_grad,step_callback=lambda b,x:print('\r beta=%f'% b,end=''))
#solspectral_bfgs_sol = optimizer.run(method='L-BFGS-B',reinitialize=False)

In [ ]:
#plt.subplot(1,2,1)
#plt.plot(range(0,len(beta_range)), [solspectral_bfgs_sol[i].rel_entropy for i in range(0,len(beta_range))])
#plt.subplot(1,2,2)
#plt.plot(range(0,len(beta_range)), [solspectral_bfgs_sol[i].DeltaL for i in range(0,len(beta_range))])

In [ ]:
# initialize from the solution
optimizer = nq.ExpectedModelOptimizer(A=Astar,beta_range=beta_range,x0=np.random.random(N))
optimizer.setup(expected_adj_fun=M,expected_lapl_grad_fun=M.expected_laplacian_grad,step_callback=lambda b,x:print('\r beta=%f'% b,end=''))
solspectral_bfgs_sol_reinit = optimizer.run(method='L-BFGS-B',reinitialize=True)

In [ ]:
plt.figure(figsize=(30,8))
plt.subplot(1,2,1)
plt.plot(range(0,len(beta_range)), [solspectral_bfgs_sol_reinit[i].rel_entropy for i in range(0,len(beta_range))],'.')
plt.subplot(1,2,2)
plt.plot(range(0,len(beta_range)), [solspectral_bfgs_sol_reinit[i].DeltaL for i in range(0,len(beta_range))],'.')
np.min([solspectral_bfgs_sol_reinit[i].DeltaL for i in range(0,len(beta_range))][100:]),np.argmin([solspectral_bfgs_sol_reinit[i].DeltaL for i in range(0,len(beta_range))][100:-1])

In [ ]:
def factor(numberToFactor, arr=list()):
    i = 2
    maximum = numberToFactor / 2 + 1
    while i < maximum:
        if numberToFactor % i == 0:
            return factor(numberToFactor/i,arr + [i])
        i += 1
    return list(set(arr + [numberToFactor]))

print(factor(195))

In [ ]:
pos = nx.spring_layout(G,iterations=25000,k=5)

In [ ]:
5*13*3

In [ ]:
fig,ax = plt.subplots(ncols=15,nrows=13,squeeze=True,figsize=(30,int(30*13/15)))
ax=ax.flatten()
Z=np.random.random([len(Astar),len(Astar)])
for i in np.arange(0,195):
    j=i
    P = (M(solspectral_bfgs_sol_reinit[i].x)>Z).astype(float)
    GP = nx.from_numpy_array(P)
    nx.draw_networkx_nodes(GP,pos=pos, node_size=50,ax=ax[j])
    nx.draw_networkx_edges(GP,pos=pos, ax=ax[j],edge_color='k',alpha=0.2)
    nx.draw_networkx_edges(G,pos=pos, ax=ax[j],edge_color='r') # draw original edges
    ax[j].set_title('beta=%f deltaL=%f' % (solspectral_bfgs_sol_reinit[i].beta,solspectral_bfgs_sol_reinit[i].DeltaL))
    ax[j].axis('off')
plt.subplots_adjust()

In [ ]:
fig,ax = plt.subplots(ncols=5,nrows=100,squeeze=True,figsize=(30,600))
ax=ax.flatten()
Z=np.random.random([len(Astar),len(Astar)])
for i in range(0,len(beta_range)):
    j=i
    P = (M(solspectral_bfgs_sol_reinit[i].x)>Z).astype(float)
    GP = nx.from_numpy_array(P)
    nx.draw_networkx_nodes(GP,pos=pos, node_size=50,ax=ax[j])
    nx.draw_networkx_edges(GP,pos=pos, ax=ax[j],edge_color='k',alpha=0.2)
    nx.draw_networkx_edges(G,pos=pos, ax=ax[j],edge_color='r') # draw original edges
    ax[j].set_title('beta=%f deltaL=%f iter=%i' % (solspectral_bfgs_sol_reinit[i].beta,solspectral_bfgs_sol_reinit[i].DeltaL,i))
    ax[j].axis('off')
plt.tight_layout()